In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Veri temizleme sırasında bazı çöp veriler ek 3 sütuna taştığı için o sütunları adlandırıp, sildik

In [2]:
csv_file_path = '../data/earthquakedata0.csv'
df = pd.read_csv(csv_file_path, sep=',', names=['Tarih', 'Saat', 'Enlem(N)', 'Boylam(E)', 'Derinlik(km)', 'MD', 'ML', 'Mw', 'Yer'])
df[['Tarih', 'Saat', 'Enlem(N)', 'Boylam(E)', 'Derinlik(km)', 'MD', 'ML', 'Mw', 'Yer','sil1','sil2','sil3']] = df['Tarih'].str.split(',', expand=True)

# Yeni bir CSV dosyasına yaz
new_csv_file_path = '../data/cleanedearthquake.csv'
df.to_csv(new_csv_file_path, index=False)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Tarih         501 non-null    object
 1   Saat          501 non-null    object
 2   Enlem(N)      501 non-null    object
 3   Boylam(E)     501 non-null    object
 4   Derinlik(km)  501 non-null    object
 5   MD            501 non-null    object
 6   ML            501 non-null    object
 7   Mw            501 non-null    object
 8   Yer           501 non-null    object
 9   sil1          103 non-null    object
 10  sil2          61 non-null     object
 11  sil3          2 non-null      object
dtypes: object(12)
memory usage: 47.1+ KB


In [4]:
df = df.drop(['Enlem(N)','Boylam(E)','MD','Mw','sil1','sil2','sil3'], axis=1)

In [5]:
df.columns

Index(['Tarih', 'Saat', 'Derinlik(km)', 'ML', 'Yer'], dtype='object')

In [6]:
df.head(10)

,Tarih,Saat,Derinlik(km),ML,Yer
0,Tarih,Saat,Derinlik(km),ML,Yer
1,2023.12.22,14:44:42,11.9,1.5,GEMLIK
2,2023.12.22,14:27:53,28.2,2.0,KARACAM-KOYCEGIZ(MUGLA)
3,2023.12.22,14:25:09,15.5,3.4,KARADENIZ
4,2023.12.22,14:01:08,5.4,1.8,FINDIK-GOKSUN(KAHRAMANMARAS)
5,2023.12.22,13:36:06,12.0,1.8,GELINPERTEK-YEDISU(BINGOL)
6,2023.12.22,13:15:38,5.0,2.4,KAPIKAYA-KOZAN(ADANA)
7,2023.12.22,13:06:15,5.0,1.1,GEMLIK
8,2023.12.22,13:04:31,9.5,2.1,GELINPERTEK-YEDISU(BINGOL)
9,2023.12.22,12:57:47,13.2,1.4,AFYONKARAHISAR(AFYONKARAHISAR)


Yer sütununda bulunan parantez dışındaki veriler çöp veriler olduğu için onları da siliyoruz.

In [7]:
df['Yer'] = df['Yer'].str.extract(r'\((.*?)\)')

In [8]:
df["Yer"]

0                NaN
1                NaN
2              MUGLA
3                NaN
4      KAHRAMANMARAS
           ...      
496              NaN
497              NaN
498              NaN
499    KAHRAMANMARAS
500    KAHRAMANMARAS
Name: Yer, Length: 501, dtype: object

Yer sütunundaki boş değere sahip olan satırları siliyoruz.

In [9]:
df.dropna(subset=['Yer'], inplace=True)

In [10]:
df["Yer"]

2              MUGLA
4      KAHRAMANMARAS
5             BINGOL
6              ADANA
8             BINGOL
           ...      
492    KAHRAMANMARAS
493           MERSIN
494            BURSA
499    KAHRAMANMARAS
500    KAHRAMANMARAS
Name: Yer, Length: 397, dtype: object

Tarih ve saat sütunundaki verileri timestamp olarak birleştiriyoruz.

In [11]:
import datetime
import time

In [12]:
timestamp=[]
for d, t in zip(df['Tarih'], df['Saat']):
    try:
        ts = datetime.datetime.strptime(d + ' ' + t, '%Y.%m.%d %H:%M:%S')
        timestamp.append(time.mktime(ts.timetuple()))
    except ValueError as e:
        print(f"Error for date {d} and time {t}: {e}")
        timestamp.append('ValueError')

timeStamp = pd.Series(timestamp)
df['Timestamp'] = timeStamp.values

In [13]:
display(df)

,Tarih,Saat,Derinlik(km),ML,Yer,Timestamp
2,2023.12.22,14:27:53,28.2,2.0,MUGLA,1.703244e+09
4,2023.12.22,14:01:08,5.4,1.8,KAHRAMANMARAS,1.703243e+09
5,2023.12.22,13:36:06,12.0,1.8,BINGOL,1.703241e+09
6,2023.12.22,13:15:38,5.0,2.4,ADANA,1.703240e+09
8,2023.12.22,13:04:31,9.5,2.1,BINGOL,1.703239e+09
...,...,...,...,...,...,...
492,2023.12.15,11:51:59,5.0,1.0,KAHRAMANMARAS,1.702630e+09
493,2023.12.15,11:45:09,0.0,1.1,MERSIN,1.702630e+09
494,2023.12.15,11:44:38,5.4,1.3,BURSA,1.702630e+09
499,2023.12.15,10:05:05,4.7,1.8,KAHRAMANMARAS,1.702624e+09


Artık "Tarih" ve "Saat" sütunu ile bir işimiz kalmadığı için onları da siliyoruz.

In [14]:
df = df.drop(['Tarih','Saat'], axis=1)

In [15]:
df=df.rename(columns = {'Derinlik(km)':'Derinlik'})

In [16]:
df["Derinlik"] = df.Derinlik.astype(float)
df["ML"] = df.ML.astype(float)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 397 entries, 2 to 500
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Derinlik   397 non-null    float64
 1   ML         397 non-null    float64
 2   Yer        397 non-null    object 
 3   Timestamp  397 non-null    float64
dtypes: float64(3), object(1)
memory usage: 15.5+ KB


In [18]:
df.to_csv(new_csv_file_path, index=False)